# Config generator for Home Assistant

In [10]:
import yaml

## Stale automations

Detect stale sensor readings and shut down associated thermostats

Example:

```yaml
- id: '1604254268934'
  alias: 'Stale sensor: Pip''s Room'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.pip_temperature.last_changed))/(60*60) > 12 }}'
    - condition: state
      entity_id: automation.turn_pip_s_heating_on
      state: 'on'
  action:
  - data: {}
    entity_id: automation.turn_pip_s_heating_on
    service: automation.turn_off
  - data:
      message: Disabling Pip Heating
      title: 'Stale sensor: pip'
    service: notify.mobile_app_pixel_2
  mode: single
```

### Simple sensors 1:1 control automation

These have a single sensor and a single automation that turns on the heating.

- When stale, turn off the automation.
- When fresh, turn on the automation.

In [18]:
# escape any ' with ''
settings = [
    {"id": "pip_room_stale", 
     "alias": "Pip''s room", 
     "sensor": "pip_temperature", 
     "timeout": 4, 
     "automation": "turn_pip_s_heating_on"},
    {"id": "kitchen_stale",
     "alias": "Kitchen", 
     "sensor": "kitchen_temperature", 
     "timeout": 4, 
     "automation": "kitchen_heating_on"},
    {"id": "hot_water_stale",
     "alias": "Hot Water", 
     "sensor": "hot_water", 
     "timeout": 2, 
     "automation": "hot_water"}
]

In [19]:
config=""
for item in settings:
    config += f"""
- id: '{item["id"]}'
  alias: 'Stale sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) > {item["timeout"]} }}}}'
    - condition: state
      entity_id: automation.{item["automation"]}
      state: 'on'
  action:
  - data: {{}}
    entity_id: automation.{item["automation"]}
    service: automation.turn_off
  - data:
      message: Disabling {item["alias"]}
      title: 'Stale sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
- id: '{item["id"]}_enable'
  alias: 'Fresh sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 8
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) < {item["timeout"]} }}}}'
    - condition: state
      entity_id: automation.{item["automation"]}
      state: 'off'
  action:
  - data: {{}}
    entity_id: automation.{item["automation"]}
    service: automation.turn_on
  - data:
      message: Enabling {item["alias"]}
      title: 'Fresh sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
    """

try:
    yaml.safe_load(config)
    print(config)
except:
    print("YAML failed to parse")


- id: 'pip_room_stale'
  alias: 'Stale sensor: Pip''s room'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.pip_temperature.last_changed))/(60*60) > 4 }}'
    - condition: state
      entity_id: automation.turn_pip_s_heating_on
      state: 'on'
  action:
  - data: {}
    entity_id: automation.turn_pip_s_heating_on
    service: automation.turn_off
  - data:
      message: Disabling Pip''s room
      title: 'Stale sensor: pip_temperature'
    service: notify.mobile_app_pixel_2
  mode: single
- id: 'pip_room_stale_enable'
  alias: 'Fresh sensor: Pip''s room'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 8
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.pip_temperature.last_changed))/(60

### Simple sensors 1:many control automation OR

These have a single sensor and two or more automations that turns on the heating.

- When stale, turn off the automations.
- When fresh, turn on the automations.

In [15]:
# escape any ' with ''
settings = [
    {"id": "attic_office_stale", 
     "alias": "Attic Office", 
     "sensor": "attic_office_temperature", 
     "timeout": 4, 
     "automations": ["attic_heating_on","attic_heating_on_f"]}  
]

In [16]:
config=""
for item in settings:
    config += f"""
- id: '{item["id"]}'
  alias: 'Stale sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) > {item["timeout"]} }}}}'
    - condition: or
    conditions:
"""
    for automation in item["automations"]:
        config += f"""
    - condition: state
      entity_id: automation.{automation}
      state: 'on'
"""
    config += f"""
  action:
"""
    for automation in item["automations"]:
        config += f"""
  - data: {{}}
    entity_id: automation.{automation}
    service: automation.turn_off
"""
    config += f"""
  - data:
      message: Disabling {item["alias"]}
      title: 'Stale sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
- id: '{item["id"]}_enable'
  alias: 'Fresh sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 8
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) < {item["timeout"]} }}}}'
    - condition: or
    conditions:
"""
    for automation in item["automations"]:
        config += f"""
    - condition: state
      entity_id: automation.{automation}
      state: 'off'
"""
    config += f"""
  action:
"""
    for automation in item["automations"]:
        config += f"""
  - data: {{}}
    entity_id: automation.{automation}
    service: automation.turn_on
"""
    config += f"""  - data:
      message: Enabling {item["alias"]}
      title: 'Fresh sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
    """

try:
    yaml.safe_load(config)
    print(config)
except:
    print("YAML failed to parse")


- id: 'attic_office_stale'
  alias: 'Stale sensor: Attic Office'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.attic_office_temperature.last_changed))/(60*60) > 4 }}'
    - condition: or
    conditions:

    - condition: state
      entity_id: automation.attic_heating_on
      state: 'on'

    - condition: state
      entity_id: automation.attic_heating_on_f
      state: 'on'

  action:

  - data: {}
    entity_id: automation.attic_heating_on
    service: automation.turn_off

  - data: {}
    entity_id: automation.attic_heating_on_f
    service: automation.turn_off

  - data:
      message: Disabling Attic Office
      title: 'Stale sensor: attic_office_temperature'
    service: notify.mobile_app_pixel_2
  mode: single
- id: 'attic_office_stale_enable'
  alias: 'Fresh sensor: Attic Office'
  description: ''
 

In [22]:
# OR/AND
config=""
for item in settings:
    config += f"""
- id: '{item["id"]}'
  alias: 'Stale sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) > {item["timeout"]} }}}}'
    - condition: or
    conditions:
"""
    for automation in item["automations"]:
        config += f"""
    - condition: state
      entity_id: automation.{automation}
      state: 'on'
"""
    config += f"""
  action:
"""
    for automation in item["automations"]:
        config += f"""
  - data: {{}}
    entity_id: automation.{automation}
    service: automation.turn_off
"""
    config += f"""
  - data:
      message: Disabling {item["alias"]}
      title: 'Stale sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
- id: '{item["id"]}_enable'
  alias: 'Fresh sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 8
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) < {item["timeout"]} }}}}'
"""
    for automation in item["automations"]:
        config += f"""
    - condition: state
      entity_id: automation.{automation}
      state: 'off'
"""
    config += f"""
  action:
"""
    for automation in item["automations"]:
        config += f"""
  - data: {{}}
    entity_id: automation.{automation}
    service: automation.turn_on
"""
    config += f"""  - data:
      message: Enabling {item["alias"]}
      title: 'Fresh sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
    """

try:
    yaml.safe_load(config)
    print(config)
except:
    print("YAML failed to parse")


- id: 'attic_office_stale'
  alias: 'Stale sensor: Attic Office'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.attic_office_temperature.last_changed))/(60*60) > 4 }}'
    - condition: or
    conditions:

    - condition: state
      entity_id: automation.attic_heating_on
      state: 'on'

    - condition: state
      entity_id: automation.attic_heating_on_f
      state: 'on'

  action:

  - data: {}
    entity_id: automation.attic_heating_on
    service: automation.turn_off

  - data: {}
    entity_id: automation.attic_heating_on_f
    service: automation.turn_off

  - data:
      message: Disabling Attic Office
      title: 'Stale sensor: attic_office_temperature'
    service: notify.mobile_app_pixel_2
  mode: single
- id: 'attic_office_stale_enable'
  alias: 'Fresh sensor: Attic Office'
  description: ''
 

### Simple sensors 1:many control automation AND

These have a single sensor and two or more automations that turns on the heating.

- When stale, turn off the automations.
- When fresh, turn on the automations.

In [20]:
# escape any ' with ''
settings = [
    {"id": "attic_office_stale", 
     "alias": "Attic Office", 
     "sensor": "attic_office_temperature", 
     "timeout": 4, 
     "automations": ["attic_heating_on","attic_heating_on_f"]}  
]

In [21]:
config=""
for item in settings:
    config += f"""
- id: '{item["id"]}'
  alias: 'Stale sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) > {item["timeout"]} }}}}'
"""
    for automation in item["automations"]:
        config += f"""
    - condition: state
      entity_id: automation.{automation}
      state: 'on'
"""
    config += f"""
  action:
"""
    for automation in item["automations"]:
        config += f"""
  - data: {{}}
    entity_id: automation.{automation}
    service: automation.turn_off
"""
    config += f"""
  - data:
      message: Disabling {item["alias"]}
      title: 'Stale sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
- id: '{item["id"]}_enable'
  alias: 'Fresh sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 8
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) < {item["timeout"]} }}}}'
"""
    for automation in item["automations"]:
        config += f"""
    - condition: state
      entity_id: automation.{automation}
      state: 'off'
"""
    config += f"""
  action:
"""
    for automation in item["automations"]:
        config += f"""
  - data: {{}}
    entity_id: automation.{automation}
    service: automation.turn_on
"""
    config += f"""  - data:
      message: Enabling {item["alias"]}
      title: 'Fresh sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
    """

try:
    yaml.safe_load(config)
    print(config)
except:
    print("YAML failed to parse")


- id: 'attic_office_stale'
  alias: 'Stale sensor: Attic Office'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.attic_office_temperature.last_changed))/(60*60) > 4 }}'

    - condition: state
      entity_id: automation.attic_heating_on
      state: 'on'

    - condition: state
      entity_id: automation.attic_heating_on_f
      state: 'on'

  action:

  - data: {}
    entity_id: automation.attic_heating_on
    service: automation.turn_off

  - data: {}
    entity_id: automation.attic_heating_on_f
    service: automation.turn_off

  - data:
      message: Disabling Attic Office
      title: 'Stale sensor: attic_office_temperature'
    service: notify.mobile_app_pixel_2
  mode: single
- id: 'attic_office_stale_enable'
  alias: 'Fresh sensor: Attic Office'
  description: ''
  trigger:
    - platform: time_patte